In [2]:
import pygrib as pg
import numpy as np
import os
import shutil

In [7]:
testpath = 'F:/Data/Graduation/fnl_months/test/'
files = os.listdir(testpath)
filelist = [testpath+filename for filename in files]

# 比较keys的年份差异

In [13]:
grb_2014 = pg.open(filelist[0])
grb_2022 = pg.open(filelist[-1])

msg_2014 = grb_2014.message(1).keys()
msg_2022 = grb_2022.message(1).keys()
print(len(msg_2014),len(msg_2022))

225 237


In [16]:
for file in filelist:
    grb = pg.open(file)
    print(len(grb.message(1).keys()))
    grb.close()

225
237
237
237
237
237
237
237
237


In [19]:
common_elements = []
for elem in msg_2022:
    if elem in msg_2014:
        common_elements.append(elem)

print("相同的元素：", common_elements)

# 比较两个列表的差异元素
difference_elements = ['2022有2014无:']
for elem in msg_2022:
    if elem not in msg_2014:
        difference_elements.append(elem)

difference_elements.append('2014有2022无:')

for elem in msg_2014:
    if elem not in msg_2022:
        difference_elements.append(elem)

print("差异的元素：", difference_elements)

相同的元素： ['globalDomain', 'GRIBEditionNumber', 'tablesVersionLatestOfficial', 'tablesVersionLatest', 'grib2divider', 'angleSubdivisions', 'missingValue', 'ieeeFloats', 'isHindcast', 'section0Length', 'identifier', 'discipline', 'editionNumber', 'totalLength', 'sectionNumber', 'section1Length', 'numberOfSection', 'centre', 'centreDescription', 'subCentre', 'tablesVersion', 'masterDir', 'localTablesVersion', 'significanceOfReferenceTime', 'year', 'month', 'day', 'hour', 'minute', 'second', 'dataDate', 'julianDay', 'dataTime', 'productionStatusOfProcessedData', 'typeOfProcessedData', 'md5Section1', 'selectStepTemplateInterval', 'selectStepTemplateInstant', 'stepType', 'is_chemical', 'is_chemical_distfn', 'is_chemical_srcsink', 'is_aerosol', 'is_aerosol_optical', 'setCalendarId', 'deleteCalendarId', 'sectionNumber', 'grib2LocalSectionPresent', 'deleteLocalDefinition', 'sectionNumber', 'gridDescriptionSectionPresent', 'section3Length', 'numberOfSection', 'sourceOfGridDefinition', 'numberOfDat

In [21]:
with open('../../data/keys.txt', 'w') as k:
    k.write('Keys in common:\n')
    for i, elem in enumerate(common_elements):
        k.write(f"{i}, {elem}\n")
    k.write('\nKeys in difference:\n')
    for i, elem in enumerate(difference_elements):
        k.write(f"{i}, {elem}\n")
k.close()

# 比较变量的年份差异

In [49]:
for file in filelist:
    grb = pg.open(file)
    print(file[-20:-9],'  ',grb.messages)
    grb.close()

20140901_00    288
20150901_00    322
20160901_00    352
20170901_00    353
20180901_00    353
20190901_00    520
20200901_00    522
20210901_00    696
20220901_00    696


In [47]:
grb_2014.message(1)['paramId']

151

In [48]:
for file in filelist:
    grb = pg.open(file)
    with open(f'../../data/msg_{file[-20:-16]}.csv','w') as f:
        for message in grb:
            output = f"{message['paramId']}, {message['name']}, {message['units']}, {message['level']}, {message['typeOfLevel']}\n"
            f.write(output)
    grb.close()

In [50]:
ids = {}
for file in filelist:
    year = file[-20:-16]
    grb = pg.open(file)
    ids[year] = [msg['name']+'_'+str(msg['level']) for msg in grb]
    grb.close()

In [53]:
with open('../../data/msg_diff.txt','w') as f:
    for year in range(2014,2022):
        f.write(f'Difference between {year} and 2022:\n')
        exist = []
        not_exist = []
        for i, id_2022 in enumerate(ids['2022']):
            try:
                j = ids[f'{year}'].index(id_2022)
                output = f"Message {i} ({id_2022}) in 2022 corresponds to {j} in {year}\n"
                exist.append(output)
            except ValueError:
                output = f"Warning: Message {i} ({id_2022}) in 2022 not corresponds to {j} in {year}\n"
                not_exist.append(output)
        f.write('Same Messages:\n')
        f.writelines(exist)
        f.write('Not Same Messages:\n')
        f.writelines(not_exist)